In [8]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import recmetrics
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [9]:
#Calculates the dot poduct subtracted by row mean
def matmult (a,amean,b,bmean):
    total = 0
    for i,j in zip(a,b):
        if (j == 0 or i==0 or np.isnan(j) or np.isnan(i) ):
            continue
        total += (i - amean)  * (j - bmean)
    return total

#Calculates the cosine simularity
def sim (a,amean,b,bmean):

    top = matmult(a,amean,b,bmean)
    bottoma = matmult(a,amean,a,amean)
    bottomb = matmult(b,bmean,b,bmean)
    if (top==0 or bottoma ==0 or bottomb==0):
        value = 0
    else:
        value = top/np.sqrt(bottoma*bottomb)
        
    return value


def check_genre(genre_list,string):
    if any(x in string for x in genre_list):
        return True
    else:
        return False


In [10]:
#The users to evaluate
evaluser_list = [51,196,256,365,591,607,672,700,751,955]

#load evaluation data
ratings_list  = pd.read_csv(r'rating.csv')
show_list = pd.read_csv(r'anime.csv')
show_list = show_list.head(5000)

#Create user list
userlist = ratings_list['user_id'].drop_duplicates().head(10000).tolist()

#Merge tables and sort movie list by volume of users that have rated it
top_list = show_list[['name','anime_id','members']].sort_values('members',ascending= False).drop('members',axis=1)
top_list = pd.merge(top_list,ratings_list[ratings_list.user_id.isin(userlist)], on='anime_id')
top_list = top_list.pivot_table(index = 'name', columns = 'user_id', values = 'rating')

#calculate the row mean
top_list = top_list.replace(-1,np.nan)
top_list = top_list.assign(rmean=lambda x: x.mean(axis=1))
top_list = top_list.assign(simval=0)

print(top_list)

user_id                                              1   2   3   4    5   6  \
name                                                                          
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hats... NaN NaN NaN NaN  NaN NaN   
&quot;Bungaku Shoujo&quot; Memoire                 NaN NaN NaN NaN  NaN NaN   
&quot;Bungaku Shoujo&quot; Movie                   NaN NaN NaN NaN  NaN NaN   
.hack//G.U. Returner                               NaN NaN NaN NaN  NaN NaN   
.hack//G.U. Trilogy                                NaN NaN NaN NaN  NaN NaN   
...                                                 ..  ..  ..  ..  ...  ..   
xxxHOLiC                                           NaN NaN NaN NaN  2.0 NaN   
xxxHOLiC Kei                                       NaN NaN NaN NaN  NaN NaN   
xxxHOLiC Movie: Manatsu no Yoru no Yume            NaN NaN NaN NaN  NaN NaN   
xxxHOLiC Rou                                       NaN NaN NaN NaN  NaN NaN   
xxxHOLiC Shunmuki                                  N

In [11]:
#Identify possible shows
ratings = pd.DataFrame(top_list['rmean'])
ratings['num of ratings'] = pd.DataFrame( top_list.drop('rmean',axis = 1).count(axis='columns'))
ratings = ratings[ratings['num of ratings']>ratings['num of ratings'].mean()]
ratings = ratings.sort_values(['rmean','num of ratings'],ascending=(False,False)).head(20)

genre_dict = pd.DataFrame(show_list[['name','genre']])
genre_dict.set_index('name',inplace=True)
filtered_dict = pd.DataFrame(show_list[show_list.name.isin(ratings.index.tolist())][['name','genre']])
filtered_dict = filtered_dict.reset_index()

output = pd.DataFrame()

for evaluser in evaluser_list:
    #Get Top 5 rated shows for user
    user_top_rated = top_list[evaluser].where(lambda x: x != 0 ).sort_values(ascending = False).head(5)
    user_rated = user_top_rated.index.tolist()
    user_top_rated = user_top_rated.index.tolist()
    
    reclist = list()
    for item in user_top_rated:
        glist = genre_dict.loc[item].values[0].split(', ')
        reclist.extend(filtered_dict[filtered_dict['genre'].apply(lambda x: check_genre(glist,str(x)))]['name'].tolist())
        
    reclist = set(reclist)
    reclist = list(reclist)
    
    #Check for already reviewed items
    for u in user_top_rated:
        if u in reclist:
            reclist.remove(u)
            
    print('\n User number ' + str(evaluser) + ' highest rated shows:')
    print (user_top_rated)
    
    final_list = pd.DataFrame(columns=['User','Name','Estimated Rating'])
    
    for evalshow in reclist:
        #Calculate the cosine simularity for each row
        a = top_list.loc[evalshow].drop(['rmean','simval'])
        amean = top_list.at[evalshow,'rmean']
        
        #Filter un ranked shows
        rowlist = top_list[top_list[evaluser]>0][evaluser].index.tolist()
        
        for row in rowlist:
            bmean = top_list.at[row,'rmean']
            if (bmean == 0):
                val = 0
            else:
                val = sim(a,amean,top_list.loc[row].drop(['rmean','simval']),bmean)
            top_list.loc[row,'simval'] = val
        
        #Using Nearest neighbor value for head selection
        pred_list = top_list[top_list[evaluser] != 0].sort_values('simval',ascending=False).head(5)
        
        n = 0
        s = 0
            
        for sx,nx in zip(pred_list[evaluser].tolist(),pred_list['simval'].tolist() ):
            if (not np.isnan(sx) and not np.isnan(nx)):
                s += sx * nx
                n += nx   
        if (n==0 or s==0):
            r = 0
        else:
            r = s/n
           
        final_list = final_list.append({'User':evaluser,'Name': evalshow,'Estimated Rating':r}, ignore_index = True)
    
    output = output.append(final_list.sort_values('Estimated Rating',ascending=False).head(5), ignore_index=True)

print('\nShow Reccomendations')
print(output)
output.to_csv('User_Reccomendations.csv')


 User number 51 highest rated shows:
['ef: A Tale of Memories.', 'Perfect Blue', 'Nana', 'Mousou Dairinin', 'Monster']

 User number 196 highest rated shows:
['Mushishi', 'Sen to Chihiro no Kamikakushi', 'Samurai Champloo', 'Tonari no Totoro', 'Hikaru no Go']

 User number 256 highest rated shows:
['Mahou Sensei Negima!', 'Sword Art Online II', 'Mahou Sensei Negima! Shiroki Tsubasa Ala Alba', 'Mahou Sensei Negima! Mou Hitotsu no Sekai Extra: Mahou Shoujo Yue', 'Mahou Sensei Negima! Mou Hitotsu no Sekai']

 User number 365 highest rated shows:
['Zoku Natsume Yuujinchou', 'Kannazuki no Miko', 'Gravitation', 'Dragon Ball Z', 'Sen to Chihiro no Kamikakushi']

 User number 591 highest rated shows:
['Nana', 'InuYasha', 'Clannad: After Story', 'Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.', 'Angel Beats!']

 User number 607 highest rated shows:
['Shingeki no Kyojin', 'Shaman King', 'Higurashi no Naku Koro ni', 'Death Note', 'Darker than Black: Kuro no Keiyakusha']

 User number 6

In [12]:
#Get count of each recommended title and print the top 10 most recommended titles based on the users being analyzed
titleCount = output['Name'].value_counts().head(10)
print("\nTop 10 Most Recommended:")
print(titleCount)


Top 10 Most Recommended:
Code Geass: Hangyaku no Lelouch R2    8
Fullmetal Alchemist: Brotherhood      7
Code Geass: Hangyaku no Lelouch       6
One Punch Man                         5
Haikyuu!! Second Season               4
Cowboy Bebop                          3
Gintama&#039;                         3
Clannad: After Story                  3
Hunter x Hunter (2011)                2
Tengen Toppa Gurren Lagann            2
Name: Name, dtype: int64


In [15]:
intra_list = {}
recmov = output['Name'].tolist()

df_atribute = list(intra_list.keys())
#df_atribute.append('name')
df_atribute = pd.DataFrame(columns = df_atribute)
for item in set(recmov):
    df_atribute = df_atribute.append(pd.Series(name = item))

for item in recmov:
    show_genre = show_list[show_list['name'] == item]['genre'].values[0].split(', ')[0]
    
    df_atribute.at[item,show_genre] = 1
    
    if show_genre in intra_list:
        intra_list[show_genre].append(item)
    else:
        intra_list[show_genre] = [item]

df_rat = []
for item in intra_list.keys():
    df_rat.append(intra_list[item])
    
df_atribute = df_atribute.fillna(0)

recmet = recmetrics.intra_list_similarity(df_rat,df_atribute)
print("Reccommended shows and their primary genre")
print(df_atribute)
print ('\nThe intra-list similarity is ' + str(recmet))

Reccommended shows and their primary genre
                                    Action  Adventure  Drama  Comedy  Sci-Fi
One Punch Man                          1.0        0.0    0.0     0.0     0.0
Haikyuu!! Second Season                0.0        0.0    0.0     1.0     0.0
Shigatsu wa Kimi no Uso                0.0        0.0    1.0     0.0     0.0
Clannad: After Story                   0.0        0.0    1.0     0.0     0.0
Sen to Chihiro no Kamikakushi          0.0        1.0    0.0     0.0     0.0
Code Geass: Hangyaku no Lelouch        1.0        0.0    0.0     0.0     0.0
Suzumiya Haruhi no Shoushitsu          0.0        0.0    0.0     1.0     0.0
Steins;Gate                            0.0        0.0    0.0     0.0     1.0
Fullmetal Alchemist: Brotherhood       1.0        0.0    0.0     0.0     0.0
Mushishi Zoku Shou 2nd Season          0.0        1.0    0.0     0.0     0.0
Tengen Toppa Gurren Lagann             1.0        0.0    0.0     0.0     0.0
Gintama&#039;                    